In [1]:
import pandas as pd

In [2]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


# Chiller Group

total_consumption_chiller_group = β1*number_active_chillers + β2*outdoor_temperature + β3*outdoor_humidity + β4*Chiller Set point + β5*Chiller Efficiency

In [16]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-03T01-15-58_industrial_site2_processed.csv')

In [17]:
df.columns

Index(['Time', '01 General_Electric_Active Energy (kWh)',
       '01 General_Electric_Active Power (kW)',
       '01 General_Electric_Power Factor (real)',
       '02 Production_Electric_Active Energy (kWh)',
       '02 Production_Electric_Active Power (kW)',
       '02 Production_Electric_Power Factor (real)',
       '03 Chiller Group_Electric_Active Energy (kWh)',
       '03 Chiller Group_Electric_Active Power (kW)',
       '03 Chiller Group_Electric_Power Factor (real)',
       '04 UTA_Electric_Active Energy (kWh)',
       '04 UTA_Electric_Active Power (kW)',
       '04 UTA_Electric_Power Factor (real)',
       '05 Compressors_Electric_Active Energy (kWh)',
       '05 Compressors_Electric_Active Power (kW)',
       '05 Compressors_Electric_Power Factor (real)',
       '06 Offices_Electric_Active Energy (kWh)',
       '06 Offices_Electric_Active Power (kW)',
       '06 Offices_Electric_Power Factor (real)',
       '07 Data Center_Electric_Active Energy (kWh)',
       '07 Data Center_

In [18]:
columns_to_include = ['Time', '03 Chiller Group_Electric_Active Energy (kWh)', 'temp', 'humidity', 'heat_index', '03 Chiller Group_Electric_Power Factor (real)']

df_for_synthesis = df[columns_to_include]

df_for_synthesis.columns

Index(['Time', '03 Chiller Group_Electric_Active Energy (kWh)', 'temp',
       'humidity', 'heat_index',
       '03 Chiller Group_Electric_Power Factor (real)'],
      dtype='object')

In [19]:
df_for_synthesis = df_for_synthesis.rename(columns={'temp': 'outdoor_temp', 'humidity': 'outdoor_humidity', '03 Chiller Group_Electric_Power Factor (real)': 'Chiller Group_Electric_Power Factor (real)(efficiency)'})

In [20]:
df_for_synthesis.head()

,Time,03 Chiller Group_Electric_Active Energy (kWh),outdoor_temp,outdoor_humidity,heat_index,Chiller Group_Electric_Power Factor (real)(efficiency)
0,7/1/22 00:00,153.12350,15.97,0.37,15.97,0.78375
1,7/1/22 01:00,152.16125,15.10,0.37,15.10,0.77975
2,7/1/22 02:00,128.90925,14.80,0.38,14.80,0.79450
3,7/1/22 03:00,123.00075,15.10,0.38,15.10,0.78075
4,7/1/22 04:00,124.18700,15.97,0.38,15.97,0.78475


In [21]:
df_for_synthesis.to_csv('data/target_variables/target_chiller_group.csv')

In [22]:
file_path = 'data/target_variables/target_chiller_group.csv'
data = pd.read_csv(file_path)
data.columns

Index(['Unnamed: 0', 'Time', '03 Chiller Group_Electric_Active Energy (kWh)',
       'outdoor_temp', 'outdoor_humidity', 'heat_index',
       'Chiller Group_Electric_Power Factor (real)(efficiency)'],
      dtype='object')

In [23]:
import numpy as np
import pandas as pd

# Define the function to calculate chiller settings
def calculate_chiller_settings(heat_index):
    # Calculate chiller set point
    if heat_index < 18:
        set_point = 18
    elif heat_index > 40:
        set_point = 25
    else:
        set_point = round(18 + (heat_index - 18) * (7 / (40 - 18)))
    
    # Calculate number of active chillers
    if heat_index < 18:
        active_chillers = 0
    else:
        # Simple model: the number of chillers increases with heat_index
        # This is a simplistic linear relationship for demonstration purposes
        heat_index_factor = (heat_index - 18) / (40 - 18)
        active_chillers = int(np.ceil(heat_index_factor * 10))  # Example scale factor, adjust as needed
        
    return set_point, active_chillers

# Assuming your existing DataFrame is named df and has columns 'outdoor_temp' and 'humidity'
# Apply the function to each row in the DataFrame
data[['set_point', 'num_active_chiller']] = data.apply(
    lambda row: pd.Series(calculate_chiller_settings(row['heat_index'])),
    axis=1
)

data.head()

,Unnamed: 0,Time,03 Chiller Group_Electric_Active Energy (kWh),outdoor_temp,outdoor_humidity,heat_index,Chiller Group_Electric_Power Factor (real)(efficiency),set_point,num_active_chiller
0,0,7/1/22 00:00,153.12350,15.97,0.37,15.97,0.78375,18,0
1,1,7/1/22 01:00,152.16125,15.10,0.37,15.10,0.77975,18,0
2,2,7/1/22 02:00,128.90925,14.80,0.38,14.80,0.79450,18,0
3,3,7/1/22 03:00,123.00075,15.10,0.38,15.10,0.78075,18,0
4,4,7/1/22 04:00,124.18700,15.97,0.38,15.97,0.78475,18,0


In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

data.drop('Unnamed: 0')
# Ensure your DataFrame 'df' has the necessary columns
# Assuming df already has 'outdoor_temp', 'outdoor_humidity', 'set_point', and 'num_active_chiller' columns

# Create the pair plot
sns.pairplot(data)
plt.show()

KeyError: "['Unnamed: 0'] not found in axis"

In [25]:
data.to_csv('data/synthesized_data/chiller_group_synthesized.csv', index=False)

# Get Coefficients

In [26]:
file_path = 'data/synthesized_data/chiller_group_synthesized.csv'
df = pd.read_csv(file_path, index_col=False)
df.columns

Index(['Unnamed: 0', 'Time', '03 Chiller Group_Electric_Active Energy (kWh)',
       'outdoor_temp', 'outdoor_humidity', 'heat_index',
       'Chiller Group_Electric_Power Factor (real)(efficiency)', 'set_point',
       'num_active_chiller'],
      dtype='object')

In [34]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump, load

# Count total NaN values in the entire DataFrame
total_nans = df.isna().sum().sum()
print(f'Total NaN values in the DataFrame: {total_nans}')

# Count NaN values in each column
nans_per_column = df.isna().sum()
print('NaN values in each column:')
print(nans_per_column)

# Count the number of rows that contain any NaN values
rows_with_nans_count = df.isna().any(axis=1).sum()
print(f'Number of rows with at least one NaN value: {rows_with_nans_count}')

# Replace NaN values with the mean in a specific column
column_to_fill = 'Chiller Group_Electric_Power Factor (real)(efficiency)'  # replace with your specific column name
df[column_to_fill].fillna(df[column_to_fill].mean(), inplace=True)

# Verify if there are any NaNs left in that column
nan_counts_after_filling = df[column_to_fill].isna().sum()
print(f'Total NaN values in the column "{column_to_fill}" after filling: {nan_counts_after_filling}')



Total NaN values in the DataFrame: 0
NaN values in each column:
Unnamed: 0                                                0
Time                                                      0
03 Chiller Group_Electric_Active Energy (kWh)             0
outdoor_temp                                              0
outdoor_humidity                                          0
heat_index                                                0
Chiller Group_Electric_Power Factor (real)(efficiency)    0
set_point                                                 0
num_active_chiller                                        0
heat_index_set_point_diff                                 0
dtype: int64
Number of rows with at least one NaN value: 0
Total NaN values in the column "Chiller Group_Electric_Power Factor (real)(efficiency)" after filling: 0


/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/ipykernel_73209/3976399992.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_to_fill].fillna(df[column_to_fill].mean(), inplace=True)


In [35]:
# Define the target and features
target = '03 Chiller Group_Electric_Active Energy (kWh)'
df['heat_index_set_point_diff'] = df['heat_index'] - df['set_point']
features = ['heat_index_set_point_diff', 'num_active_chiller', 'Chiller Group_Electric_Power Factor (real)(efficiency)']

# Split the data into features (X) and target (y)
X = df[features]
y = df[target]

# Initialize the linear regression model
model = LinearRegression()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')

# Fit the model on the entire dataset
model.fit(X, y)

# Save the trained model to a file
model_filename = 'equation_modeling/models/chiller_regression_model.joblib'
dump(model, model_filename)

print(f'Mean Squared Error (Cross-Validation): {-cv_results.mean()}')
print(f'Standard Deviation (Cross-Validation): {cv_results.std()}')

# Optional: Print the coefficients of the model
coefficients = pd.DataFrame(model.coef_, features, columns=['Coefficient'])
print(coefficients)

# Loading the model (for demonstration purposes)
# loaded_model = load(model_filename)
# print(f'Loaded Model Coefficients: {loaded_model.coef_}')

Mean Squared Error (Cross-Validation): 935.1390702085619
Standard Deviation (Cross-Validation): 27.224554300148473
                                                    Coefficient
heat_index_set_point_diff                              3.357534
num_active_chiller                                     0.583621
Chiller Group_Electric_Power Factor (real)(effi...    28.143946


In [37]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump

# Define the target and features
target = '03 Chiller Group_Electric_Active Energy (kWh)'
df['heat_index_set_point_diff'] = df['heat_index'] - df['set_point']
features = ['heat_index_set_point_diff', 'Chiller Group_Electric_Power Factor (real)(efficiency)']

# Known coefficient for num_active_chiller
known_coefficient = 39
num_active_chiller = df['num_active_chiller']

# Adjust the target variable
y_adjusted = df[target] - (known_coefficient * num_active_chiller)

# Split the data into features (X) and adjusted target (y_adjusted)
X = df[features]

# Initialize the linear regression model
model = LinearRegression()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results_mse = cross_val_score(model, X, y_adjusted, cv=kfold, scoring='neg_mean_squared_error')
cv_results_r2 = cross_val_score(model, X, y_adjusted, cv=kfold, scoring='r2')

# Fit the model on the entire dataset
model.fit(X, y_adjusted)

# Get the intercept and coefficients
intercept = model.intercept_
coefficients = model.coef_

# Save the trained model to a file
model_params = {
    'intercept': intercept,
    'coefficients': coefficients,
    'known_coefficient': known_coefficient,
    'features': features
}
model_filename = 'equation_modeling/models/constrained_linear_regression_model.joblib'
dump(model_params, model_filename)

# Calculate and print performance metrics
def predict(X, model_params, num_active_chiller):
    intercept = model_params['intercept']
    coefficients = model_params['coefficients']
    known_coefficient = model_params['known_coefficient']
    return intercept + X.dot(coefficients) + known_coefficient * num_active_chiller

y_pred_adjusted = model.predict(X)
y_pred = y_pred_adjusted + (known_coefficient * num_active_chiller)
mse = mean_squared_error(df[target], y_pred)
r2 = r2_score(df[target], y_pred)

print(f'Mean Squared Error (Cross-Validation): {-cv_results_mse.mean()}')
print(f'Standard Deviation (Cross-Validation): {cv_results_mse.std()}')
print(f'R^2 Score (Cross-Validation): {cv_results_r2.mean()}')

# Optional: Print the coefficients of the model
coefficients_df = pd.DataFrame(coefficients, features, columns=['Coefficient'])
print(f'Intercept: {intercept}')
print(f'Known Coefficient for num_active_chiller: {known_coefficient}')
print(coefficients_df)

# Function to make predictions using the saved model
def predict(X, model_params, num_active_chiller):
    intercept = model_params['intercept']
    coefficients = model_params['coefficients']
    known_coefficient = model_params['known_coefficient']
    return intercept + X.dot(coefficients) + known_coefficient * num_active_chiller

# Optional: Load the model and make predictions
# loaded_model_params = load(model_filename)
# predictions = predict(X, loaded_model_params, num_active_chiller)
# print(f'Model predictions: {predictions}')

Mean Squared Error (Cross-Validation): 3476.446268397296
Standard Deviation (Cross-Validation): 95.33496490705659
R^2 Score (Cross-Validation): 0.5176265000930724
Overall R^2 Score: -0.6780849647848759
Intercept: -59.89342023912777
Known Coefficient for num_active_chiller: 39
                                                    Coefficient
heat_index_set_point_diff                             -7.987539
Chiller Group_Electric_Power Factor (real)(effi...     6.833090


In [38]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump

# Define the target and features
target = '03 Chiller Group_Electric_Active Energy (kWh)'
df['heat_index_set_point_diff'] = df['heat_index'] - df['set_point']
df['efficiency_adjusted_power'] = df['Chiller Group_Electric_Power Factor (real)(efficiency)']

# Calculate the adjusted energy consumption incorporating assumed COP
assumed_COP = 3.0  # Assumed value for COP
df['adjusted_energy'] = df[target] * df['efficiency_adjusted_power'] * assumed_COP

# Known coefficient for num_active_chiller
known_coefficient = 39
num_active_chiller = df['num_active_chiller']

# Adjust the target variable
y_adjusted = df['adjusted_energy'] - (known_coefficient * num_active_chiller)

# Define the features
features = ['heat_index_set_point_diff', 'efficiency_adjusted_power']

# Split the data into features (X) and adjusted target (y_adjusted)
X = df[features]

# Initialize the linear regression model
model = LinearRegression()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results_mse = cross_val_score(model, X, y_adjusted, cv=kfold, scoring='neg_mean_squared_error')
cv_results_r2 = cross_val_score(model, X, y_adjusted, cv=kfold, scoring='r2')

# Fit the model on the entire dataset
model.fit(X, y_adjusted)

# Get the intercept and coefficients
intercept = model.intercept_
coefficients = model.coef_

# Save the trained model to a file
model_params = {
    'intercept': intercept,
    'coefficients': coefficients,
    'known_coefficient': known_coefficient,
    'features': features
}
model_filename = 'equation_modeling/models/constrained_linear_regression_model.joblib'
dump(model_params, model_filename)

# Calculate and print performance metrics
y_pred_adjusted = model.predict(X)
y_pred = y_pred_adjusted + (known_coefficient * num_active_chiller)
mse = mean_squared_error(df[target], y_pred)
r2 = r2_score(df[target], y_pred)

print(f'Mean Squared Error (Cross-Validation): {-cv_results_mse.mean()}')
print(f'Standard Deviation (Cross-Validation): {cv_results_mse.std()}')
print(f'R^2 Score (Cross-Validation): {cv_results_r2.mean()}')

# Optional: Print the coefficients of the model
coefficients_df = pd.DataFrame(coefficients, features, columns=['Coefficient'])
print(f'Intercept: {intercept}')
print(f'Known Coefficient for num_active_chiller: {known_coefficient}')
print(coefficients_df)

# Function to make predictions using the saved model
def predict(X, model_params, num_active_chiller):
    intercept = model_params['intercept']
    coefficients = model_params['coefficients']
    known_coefficient = model_params['known_coefficient']
    return intercept + X.dot(coefficients) + known_coefficient * num_active_chiller

# Optional: Load the model and make predictions
# loaded_model_params = load(model_filename)
# predictions = predict(X, loaded_model_params, num_active_chiller)
# print(f'Model predictions: {predictions}')

Mean Squared Error (Cross-Validation): 7527.3089418538
Standard Deviation (Cross-Validation): 262.398781379355
R^2 Score (Cross-Validation): 0.21298445668303484
Intercept: -25.265637588263914
Known Coefficient for num_active_chiller: 39
                           Coefficient
heat_index_set_point_diff    -3.683991
efficiency_adjusted_power    39.397205


In [39]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump

# Define the target and features
target = '03 Chiller Group_Electric_Active Energy (kWh)'
df['heat_index_set_point_diff'] = df['heat_index'] - df['set_point']
df['efficiency_adjusted_power'] = df['Chiller Group_Electric_Power Factor (real)(efficiency)']

# Calculate the adjusted energy consumption incorporating assumed COP
assumed_COP = 3.0  # Assumed value for COP
df['adjusted_energy'] = df[target] * df['efficiency_adjusted_power'] * assumed_COP

# Define the features
features = ['heat_index_set_point_diff', 'efficiency_adjusted_power', 'num_active_chiller']

# Split the data into features (X) and target (y)
X = df[features]
y = df['adjusted_energy']

# Initialize the linear regression model
model = LinearRegression()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results_mse = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
cv_results_r2 = cross_val_score(model, X, y, cv=kfold, scoring='r2')

# Fit the model on the entire dataset
model.fit(X, y)

# Get the intercept and coefficients
intercept = model.intercept_
coefficients = model.coef_

# Save the trained model to a file
model_params = {
    'intercept': intercept,
    'coefficients': coefficients,
    'features': features
}
model_filename = 'equation_modeling/models/constrained_linear_regression_model.joblib'
dump(model_params, model_filename)

# Calculate and print performance metrics
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error (Cross-Validation): {-cv_results_mse.mean()}')
print(f'Standard Deviation (Cross-Validation): {cv_results_mse.std()}')
print(f'R^2 Score (Cross-Validation): {cv_results_r2.mean()}')

# Optional: Print the coefficients of the model
coefficients_df = pd.DataFrame(coefficients, features, columns=['Coefficient'])
print(f'Intercept: {intercept}')
print(coefficients_df)

# Function to make predictions using the saved model
def predict(X, model_params):
    intercept = model_params['intercept']
    coefficients = model_params['coefficients']
    return intercept + X.dot(coefficients)

# Optional: Load the model and make predictions
# loaded_model_params = load(model_filename)
# predictions = predict(X, loaded_model_params)
# print(f'Model predictions: {predictions}')

Mean Squared Error (Cross-Validation): 5267.588711769505
Standard Deviation (Cross-Validation): 207.1512542532173
R^2 Score (Cross-Validation): 0.507894252348674
Intercept: 55.85081993516833
                           Coefficient
heat_index_set_point_diff     7.018069
efficiency_adjusted_power    59.500208
num_active_chiller            2.760977


In [ ]:
data['Chiller Set point'].value_counts()

In [ ]:
data['number_active_chillers'].value_counts()

# Equations

	1.	Total Consumption (total_consumption_chiller_group):

\text{total_consumption_chiller_group} = \beta_1 \cdot \text{number_active_chillers} + \beta_2 \cdot \text{outdoor_temp} + \beta_3 \cdot \text{outdoor_humidity} + \beta_4 \cdot \text{Chiller Set point} + \beta_5 \cdot \text{Chiller Efficiency}

	2.	Chiller Set Point (chiller_set_point):

\text{chiller_set_point} = \gamma_1 \cdot \text{number_active_chillers} + \gamma_2 \cdot \text{outdoor_temp} + \gamma_3 \cdot \text{outdoor_humidity} + \gamma_4 \cdot \text{Chiller Efficiency}

	3.	Number of Active Chillers (number_active_chillers):

\text{number_active_chillers} = \delta_1 \cdot \text{outdoor_temp} + \delta_2 \cdot \text{outdoor_humidity} + \delta_3 \cdot \text{Chiller Efficiency}



In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals

# Initialize the Pyomo model
model = ConcreteModel()

# Define the variables
model.number_active_chillers = Var(within=NonNegativeReals)
model.chiller_set_point = Var(within=NonNegativeReals)
model.total_consumption_chiller_group = Var(within=NonNegativeReals)

# Define the parameters (coefficients from the regression models)
beta1, beta2, beta3, beta4, beta5 = 1.2, 0.8, 0.5, 1.0, 0.9
gamma1, gamma2, gamma3, gamma4 = 0.6, 0.3, 0.2, 0.4
delta1, delta2, delta3 = 0.5, 0.3, 0.2

# Define the constants (example values, replace with actual data or parameters as needed)
outdoor_temp = 15.97
outdoor_humidity = 0.37
chiller_efficiency = 0.775

# Define the constraints
model.total_consumption_constraint = Constraint(
    expr = model.total_consumption_chiller_group == 
           beta1 * model.number_active_chillers +
           beta2 * outdoor_temp +
           beta3 * outdoor_humidity +
           beta4 * model.chiller_set_point +
           beta5 * chiller_efficiency
)

model.chiller_set_point_constraint = Constraint(
    expr = model.chiller_set_point == 
           gamma1 * model.number_active_chillers +
           gamma2 * outdoor_temp +
           gamma3 * outdoor_humidity +
           gamma4 * chiller_efficiency
)

model.number_active_chillers_constraint = Constraint(
    expr = model.number_active_chillers == 
           delta1 * outdoor_temp +
           delta2 * outdoor_humidity +
           delta3 * chiller_efficiency
)

# Define the objective function (example, can be changed based on actual requirements)
model.obj = Objective(expr=model.total_consumption_chiller_group, sense=1)

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)

# Display the results
print(f'Number of Active Chillers: {model.number_active_chillers.value}')
print(f'Chiller Set Point: {model.chiller_set_point.value}')
print(f'Total Consumption Chiller Group: {model.total_consumption_chiller_group.value}')